#### Preparar Ambiente

In [1]:
import sys
!{sys.executable} -m pip install PyAthena

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
!{sys.executable} -m pip install pytz

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


#### Conectar con pyathena

In [3]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [4]:
from datetime import datetime, timedelta
import numpy as np

#### Crear tablas por pais

In [5]:
paises = ['BO','EC','CL','VE','AR','CR','DO','SV','GT','HN','NI','PA','PY','PE','UY','BZ','GY','JM','TT','CO','MX','BR']
len(paises)

22

In [6]:
pais = 'LATAM'

In [7]:
id_city = 'id_city'

#### Crear matriz origen-destino final

In [8]:
start_time = time.time()
tabla_matrizOD_cities_pais = f'historico_mod_cities_{pais}'
tabla_matrizOD_cities_pais_final = f'historico_mod_cities_{pais}_final'
query = f'''
    CREATE TABLE IF NOT EXISTS {tabla_matrizOD_cities_pais_final} 
    WITH ( external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-public/mod/{pais}/{tabla_matrizOD_cities_pais_final}', 
                        format = 'PARQUET', 
                        parquet_compression = 'SNAPPY') AS
    SELECT *
    FROM {tabla_matrizOD_cities_pais}
    '''
cursor.execute(query)
print((time.time()-start_time)/60)
print(f'Generacion de tabla {tabla_matrizOD_cities_pais_final}: terminado')

0.07417440017064413
Generacion de tabla historico_mod_cities_LATAM_final: terminado


In [9]:
tabla_matrizOD_cities_pais_final = f'historico_mod_cities_{pais}_final'
query = f'''
    SELECT count(*) as n,
    COUNT(distinct (id_city)) as n_id_city
    FROM {tabla_matrizOD_cities_pais_final}
    '''
cursor.execute(query).as_pandas()

,n,n_id_city
0,1936209,1001


In [10]:
tabla_matrizOD_cities_pais_final = f'historico_mod_cities_{pais}_final'
query = f'''
    SELECT *
    FROM {tabla_matrizOD_cities_pais_final}
    LIMIT 10
    '''
df = cursor.execute(query).as_pandas()

In [11]:
df

,year,month,day,id_city,id_cityh,n
0,2020,2,8,249.0,255.0,18
1,2020,2,8,1303.0,1267.0,18
2,2020,2,8,720.0,720.0,18
3,2020,2,8,1070.0,1080.0,18
4,2020,2,8,1105.0,1069.0,18
5,2020,2,8,178.0,350.0,18
6,2020,2,8,207.0,225.0,18
7,2020,2,8,350.0,254.0,18
8,2020,2,8,1304.0,1302.0,18
9,2020,2,8,154.0,66.0,18
